In [2]:
import pandas as pd
import os

In [3]:
Desktop = os.path.join(os.path.expanduser("~"), "Desktop")

filepath_root = os.path.join(Desktop, "LBP_Revision")

os.path.exists(filepath_root)

path_inclusion_crit_LBP = os.path.join(filepath_root,'inclusion_codes_LBP.xlsx')

path_inclusion_crit_DJD = os.path.join(filepath_root,'inclusion_codes_DJD.xlsx')


In [4]:
def get_cpts_fromfile(filepath=''):
    df_cpt = pd.read_excel(filepath, dtype=str, sheet_name='CPT Codes')
    return df_cpt['CPT Code']       

def get_icds_fromfile(filepath=''):        
    df_icd = pd.read_excel(filepath,dtype=str, sheet_name='ICD Codes')
    return df_icd['ICD Code']

Group 1: LBP (including all)
Group 2: LBP & DJD --> exclude everything else
Group 3: LBP & Fracture --> exclude everything else
Group 4: LBP & Other(other + scoliosis) --> exclude everything else
Group 5: LBP only
Group 6: None

The python files have created file that includes every data point
What is needed to be done: Find EMPI numbers from other files and exclude them

Logic: LBP_and_DJD - (Fracture + Other + Scoliosis)

Fracture + Other = all the EMPI from fracture_all.xlsx + other_all.xlsx + scoliosis.xlsx

Group 1: LBP_filtered(version of removed duplicates) (DONE)
Group 2: 

In [ ]:
LBP_DJD_filename = 'LBP_DJD_patients'
LBP_fracture_filename = 'LBP_fracture_patients'
LBP_other_filename = 'LBP_other_patients'
#LBP_scoliosis_filename = 'LBP_scoliosis_patients'

LBP_all_filename = 'LBP_patients'
DJD_all_filename = 'DJD_patients'
fracture_all_filename = 'fracture_patients'
other_all_filename = 'other_patients'
scoliosis_all_filename = 'scoliosis_patients'


LBP_all_data = pd.read_csv(filepath_root + '/' + LBP_all_filename + '.csv', encoding = 'utf8',dtype=str)
DJD_all_data = pd.read_csv(filepath_root + '/' + DJD_all_filename + '.csv', encoding = 'utf8',dtype=str)
fracture_all_data = pd.read_csv(filepath_root + '/' + fracture_all_filename + '.csv', encoding = 'utf8',dtype=str)
other_all_data = pd.read_csv(filepath_root + '/' + other_all_filename + '.csv', encoding = 'utf8',dtype=str)
scoliosis_all_data = pd.read_csv(filepath_root + '/' + scoliosis_all_filename + '.csv', encoding = 'utf8',dtype=str)


LBP_DJD_both_data = pd.read_csv(filepath_root + '/' + LBP_DJD_filename + '.csv', encoding = 'utf8',dtype=str)
LBP_fracture_both_data = pd.read_csv(filepath_root + '/' + LBP_fracture_filename + '.csv', encoding = 'utf8',dtype=str)
LBP_other_both_data = pd.read_csv(filepath_root + '/' + LBP_other_filename + '.csv', encoding = 'utf8',dtype=str)
#LBP_scoliosis_both_data = pd.read_csv(filepath_root + '/' + LBP_scoliosis_filename + '.csv', encoding = 'utf8',dtype=str)

In [6]:
print(LBP_all_data.keys())


Index(['EMPI', 'MGH_MRN', 'Date.prc', 'Date.dia', 'Code_Type.prc', 'Code.prc',
       'Code_Type.dia', 'Code.dia', 'Diagnosis_Name', 'Procedure_Name',
       'Gender_Legal_Sex', 'Age', 'Language', 'Race1', 'Marital_status',
       'Religion', 'Date_Of_Death', 'AgeAtPrc', 'Admit_Date', 'Discharge_Date',
       'Discharge_Disposition', 'diagnosis_list', 'Diagnosis_list_combined',
       'Diagnosis_list_no_NA', 'City', 'State', 'Zip', 'Country',
       'Insurance_1', 'Insurance_2', 'Insurance_3', 'Phone_Code',
       'Physical_Date', 'BMI', 'Height', 'Weight', 'Smoking', 'Alcohol_Use',
       'Helper'],
      dtype='object')


In [7]:
LBP_diagcode = LBP_all_data[['EMPI','Age','Race1','BMI','Code.prc','Code.dia','Helper']]
LBP_diagcode['Diagnosis_combined'] = str(0)
LBP_diagcode['LBP?'] = str(0)
print(LBP_diagcode.keys())

LBP_diagcode_reference = LBP_diagcode



Index(['EMPI', 'Age', 'Race1', 'BMI', 'Code.prc', 'Code.dia', 'Helper',
       'Diagnosis_combined', 'LBP?'],
      dtype='object')


/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_87902/3317639091.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_diagcode['Diagnosis_combined'] = str(0)
/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_87902/3317639091.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_diagcode['LBP?'] = str(0)


In [8]:
LBP_diagcode.loc[0,'Diagnosis_combined'] = LBP_diagcode.loc[0,'Code.dia']
for i in range(1,len(LBP_diagcode)):
    if (LBP_diagcode.loc[i,'EMPI'] == LBP_diagcode.loc[i-1,'EMPI']):
        if LBP_diagcode.loc[i,'Code.dia'] != LBP_diagcode.loc[i-1,'Code.dia']:
            LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i,'Code.dia'] + ', ' + str(LBP_diagcode.loc[i-1,'Diagnosis_combined'])
            continue;
        else: 
            LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i-1,'Diagnosis_combined']
            continue;
    else: 
        LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i,'Code.dia']
        continue;

In [9]:
LBP_filtered = LBP_diagcode[LBP_diagcode['Helper'].notna()]
LBP_filtered_save = LBP_filtered

In [10]:
#CPT Code
#LBP only
incl_cpts_LBP = get_cpts_fromfile(path_inclusion_crit_LBP)
incl_cpts_text_LBP = '|'.join(incl_cpts_LBP.str.strip())

#DJD
incl_cpts_DJD = get_cpts_fromfile(path_inclusion_crit_DJD)
incl_cpts_text_DJD = '|'.join(incl_cpts_DJD.str.strip())

#ICD Code
#LBP only
incl_icds_LBP = get_icds_fromfile(path_inclusion_crit_LBP)
incl_icds_text_LBP = '|'.join(incl_icds_LBP.str.strip())

#DJD
incl_icds_DJD = get_icds_fromfile(path_inclusion_crit_DJD)
incl_icds_text_DJD = '|'.join(incl_icds_DJD.str.strip())

In [11]:
temp_df_LBP = pd.DataFrame
temp_df_LBP = LBP_filtered['Diagnosis_combined'].str.contains(incl_icds_text_LBP, regex=True)

LBP_filtered.index = range(len(LBP_filtered.index))

for i in range(len(temp_df_LBP)):
    if temp_df_LBP.iloc[i] == True:
        LBP_filtered.loc[i,'LBP?'] = 'LBP'

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_87902/484879454.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_filtered.loc[i,'LBP?'] = 'LBP'


In [12]:
DJD_diagcode = DJD_all_data[['EMPI','Age','Race1','BMI','Code.prc','Code.dia','Helper']]
DJD_diagcode['Diagnosis_combined'] = str(0)
DJD_diagcode['DJD?'] = str(0)
print(DJD_diagcode.keys())

DJD_diagcode_reference = DJD_diagcode

Index(['EMPI', 'Age', 'Race1', 'BMI', 'Code.prc', 'Code.dia', 'Helper',
       'Diagnosis_combined', 'DJD?'],
      dtype='object')


/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_87902/2035728344.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DJD_diagcode['Diagnosis_combined'] = str(0)
/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_87902/2035728344.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DJD_diagcode['DJD?'] = str(0)


In [13]:
DJD_diagcode.loc[0,'Diagnosis_combined'] = DJD_diagcode.loc[0,'Code.dia']
for i in range(1,len(DJD_diagcode)):
    if (DJD_diagcode.loc[i,'EMPI'] == DJD_diagcode.loc[i-1,'EMPI']):
        if DJD_diagcode.loc[i,'Code.dia'] != DJD_diagcode.loc[i-1,'Code.dia']:
            DJD_diagcode.loc[i,'Diagnosis_combined'] = DJD_diagcode.loc[i,'Code.dia'] + ', ' + str(DJD_diagcode.loc[i-1,'Diagnosis_combined'])
            continue;
        else: 
            DJD_diagcode.loc[i,'Diagnosis_combined'] = DJD_diagcode.loc[i-1,'Diagnosis_combined']
            continue;
    else: 
        DJD_diagcode.loc[i,'Diagnosis_combined'] = DJD_diagcode.loc[i,'Code.dia']
        continue;

In [14]:
DJD_filtered = DJD_diagcode[DJD_diagcode['Helper'].notna()]
DJD_filtered_save = DJD_filtered

In [15]:
temp_df_DJD = pd.DataFrame
temp_df_DJD = DJD_filtered['Diagnosis_combined'].str.contains(incl_icds_text_DJD, regex=True)

DJD_filtered.index = range(len(DJD_filtered.index))

for i in range(len(temp_df_DJD)):
    if temp_df_DJD.iloc[i] == True:
        DJD_filtered.loc[i,'DJD?'] = 'DJD'

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_87902/3061589593.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DJD_filtered.loc[i,'DJD?'] = 'DJD'


In [16]:
LBP_and_DJD = pd.concat([LBP_filtered,DJD_filtered])

In [17]:
LBP_and_DJD.index = range(len(LBP_and_DJD.index))
LBP_and_DJD['Helper'] = str(0)

In [18]:
LBP_and_DJD.sort_values(by=['EMPI'], inplace=True, ignore_index=True)
LBP_and_DJD_copy = LBP_and_DJD
LBP_and_DJD.head(10)

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,DJD?
0,100000152,81,White,27.02,72100,M51.86,0,M51.86,NaN,DJD
1,100000152,81,White,27.02,72100,G89.29,0,G89.29,LBP,NaN
2,100000243,92,Declined,24.73,72040,M54.9,0,M54.9,LBP,NaN
3,100000623,85,White,21.27,72040,M48.061,0,M48.061,NaN,DJD
4,100001760,88,White,NaN,72040,G89.29,0,G89.29,LBP,NaN
5,100002085,89,White,27.92,72100,M48.06,0,M48.06,NaN,DJD
6,100002439,89,Black,25.08,72100,M47.816,0,M47.816,NaN,DJD
7,100002524,98,White,29.76,72100,M47.817,0,M47.817,NaN,DJD
8,100003506,96,Declined,20.06,72100,M54.9,0,M54.9,LBP,NaN
9,100003506,96,Declined,20.06,72100,M47.816,0,M47.816,NaN,DJD


In [19]:
for i in range(1,len(LBP_and_DJD)):
    if LBP_and_DJD.loc[i,'EMPI'] == LBP_and_DJD.loc[i-1,'EMPI']:
        if LBP_and_DJD.loc[i,'Diagnosis_combined'] != LBP_and_DJD.loc[i-1,'Diagnosis_combined']:
            LBP_and_DJD.loc[i,'Diagnosis_combined'] = LBP_and_DJD.loc[i,'Diagnosis_combined'] + ', ' + str(LBP_and_DJD.loc[i-1,'Diagnosis_combined'])
        if LBP_and_DJD.loc[i,'LBP?'] != 'LBP':
            if LBP_and_DJD.loc[i-1,'LBP?'] == 'LBP':
                LBP_and_DJD.loc[i,'LBP?'] = 'LBP'
        if LBP_and_DJD.loc[i,'DJD?'] != 'DJD':
            if LBP_and_DJD.loc[i-1,'DJD?'] == 'DJD':
                LBP_and_DJD.loc[i,'DJD?'] = 'DJD'
    else:
        LBP_and_DJD.loc[i-1,'Helper'] = 'useful'

In [20]:
LBPnDJD_filtered = LBP_and_DJD.loc[LBP_and_DJD['Helper'] == 'useful']
LBPnDJD_filtered.reset_index(drop =True, inplace=True)
LBPnDJD_filtered['Both'] = str(0)

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_87902/3284713238.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBPnDJD_filtered['Both'] = str(0)


In [21]:
for i in range(0,len(LBPnDJD_filtered)):
    if LBPnDJD_filtered.loc[i,'LBP?'] == 'LBP':
        if LBPnDJD_filtered.loc[i,'DJD?'] == 'DJD':
            LBPnDJD_filtered.loc[i,'Both'] = 'Both'

In [22]:
LBPnDJD_filtered.head(20)

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,DJD?,Both
0,100000152,81,White,27.02,72100,G89.29,useful,"G89.29, M51.86",LBP,DJD,Both
1,100000243,92,Declined,24.73,72040,M54.9,useful,M54.9,LBP,NaN,0
2,100000623,85,White,21.27,72040,M48.061,useful,M48.061,NaN,DJD,0
3,100001760,88,White,NaN,72040,G89.29,useful,G89.29,LBP,NaN,0
4,100002085,89,White,27.92,72100,M48.06,useful,M48.06,NaN,DJD,0
5,100002439,89,Black,25.08,72100,M47.816,useful,M47.816,NaN,DJD,0
6,100002524,98,White,29.76,72100,M47.817,useful,M47.817,NaN,DJD,0
7,100003506,96,Declined,20.06,72100,M47.816,useful,"M47.816, M54.9",LBP,DJD,Both
8,100004276,92,Other,21.15,72100,M54.9,useful,"M54.9, M47.816, G89.29",LBP,DJD,Both
9,100004610,98,White,NaN,72100,M47.816,useful,"M47.816, S30.0XXA",LBP,DJD,Both


In [23]:
LBP_only = LBPnDJD_filtered.loc[LBPnDJD_filtered['LBP?'] == 'LBP']
LBP_only = LBP_only[LBP_only['Both'] != 'Both']
DJD_only = LBPnDJD_filtered.loc[LBPnDJD_filtered['DJD?'] == 'DJD']
DJD_only = DJD_only[DJD_only['Both'] != 'Both']
LBPDJD_Both_only = LBPnDJD_filtered.loc[LBPnDJD_filtered['Both'] == 'Both']

In [24]:
LBP_and_DJD_total = pd.concat([LBP_diagcode,DJD_diagcode])
LBP_and_DJD_total.drop_duplicates(subset=['EMPI'],inplace=True)
len(LBP_and_DJD_total)

5874

In [25]:
print('Number of Total Patients(LBP+DJD):  ' + str(len(LBP_and_DJD_total)-1))
print('Number of LBP only:  ' + str(len(LBP_only)))
print('Number of DJD only:  ' + str(len(DJD_only)))
print('Number of Both only:  ' + str(len(LBPDJD_Both_only)))

#LBP_filtered = LBP
#DJD_filtered = DJD
#LBP & DJD filtered = LBPnDJD_filtered


Number of Total Patients(LBP+DJD):  5873
Number of LBP only:  1477
Number of DJD only:  1553
Number of Both only:  2843


In [26]:
LBP_patients = LBP_filtered.sort_values(by=['EMPI'], ignore_index=True)
LBP_patients.head()

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?
0,100000152,81,White,27.02,72100,G89.29,useful,G89.29,LBP
1,100000243,92,Declined,24.73,72040,M54.9,useful,M54.9,LBP
2,100001760,88,White,NaN,72040,G89.29,useful,G89.29,LBP
3,100003506,96,Declined,20.06,72100,M54.9,useful,M54.9,LBP
4,100004276,92,Other,17.51,72040,G89.29,useful,G89.29,LBP


In [29]:
DJD_csv_path = os.path.join(filepath_root,'LBP_DJD_patients.csv')
LBPDJD_Both_only.to_csv(DJD_csv_path)

In [28]:
LBPDJD_Both_only.head()

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,DJD?,Both
0,100000152,81,White,27.02,72100,G89.29,useful,"G89.29, M51.86",LBP,DJD,Both
7,100003506,96,Declined,20.06,72100,M47.816,useful,"M47.816, M54.9",LBP,DJD,Both
8,100004276,92,Other,21.15,72100,M54.9,useful,"M54.9, M47.816, G89.29",LBP,DJD,Both
9,100004610,98,White,NaN,72100,M47.816,useful,"M47.816, S30.0XXA",LBP,DJD,Both
11,100006144,78,Black,23.81,72020,M47.12,useful,"M47.12, G89.29, M48.062, M48.061, M48.062, M54...",LBP,DJD,Both


Things needed to be done:
1. separate LBP only from LBP filtered --> need to sort out the ones that have DJD
    1. Find EMPI in LBPDJD_Both_only --> remove rows with this EMPI
    2. Create a CSV file